In [1]:
import pandas as pd
import numpy as np

In [6]:
prices = pd.read_parquet('data/msci_prices.parquet')
prices['Date'] = pd.to_datetime(prices['Date'])
prices = prices.set_index('Date')

In [30]:
prices[prices.index>"2024-12-31"]

,1 HK Equity,10 HK Equity,101 HK Equity,1031 HK Equity,1038 HK Equity,11 HK Equity,1113 HK Equity,1128 HK Equity,12 HK Equity,127 HK Equity,...,ZG US Equity,ZI US Equity,ZIM US Equity,ZION US Equity,ZM US Equity,ZNGA US Equity,ZOT SM Equity,ZS US Equity,ZTS US Equity,ZURN SE Equity
Date,,,,,,,,,,,,,,,,,,,,,
2025-01-02,14.196267,2.902989,2.009074,NaN,18.991754,37.108171,5.526122,1.102842,6.621731,0.861814,...,68.271936,10.190132,34.072934,71.411319,79.475236,NaN,NaN,176.973172,168.512048,NaN
2025-01-03,14.162706,2.866950,1.953789,NaN,18.917553,37.028127,5.526059,1.093450,6.618490,0.864895,...,69.265140,10.317013,32.735961,72.189211,80.227270,NaN,NaN,180.283384,168.525206,1286.409611
2025-01-06,14.111742,2.857248,1.962356,NaN,18.980464,36.198612,5.538591,1.103923,6.570722,0.830031,...,69.437160,10.085952,30.974339,71.785836,79.244016,NaN,NaN,181.614504,169.839062,1288.766901
2025-01-07,14.094299,2.849010,1.975120,NaN,18.788537,36.374049,5.512764,1.133878,6.503174,0.826863,...,67.377857,9.806394,31.212670,72.634627,77.803195,NaN,NaN,177.849690,168.189221,1296.605925
2025-01-08,13.945010,2.842530,1.975037,NaN,18.634738,35.983708,5.539212,1.115164,6.371410,0.835394,...,66.150766,9.313051,30.128498,72.088521,77.431090,NaN,NaN,181.493048,170.114275,1286.280750
2025-01-09,13.950172,2.841553,1.984736,NaN,18.714914,36.041467,5.565716,1.114897,6.367392,0.836735,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1270.430212
2025-01-10,13.954187,2.814009,1.969745,NaN,18.874002,36.146617,5.570331,1.109038,6.378601,0.847330,...,66.745718,9.204623,27.958854,70.584588,77.560906,NaN,NaN,185.439478,169.577905,1250.548784
2025-01-13,13.763982,2.797325,1.933693,NaN,18.184310,35.930757,5.517281,1.106973,6.264186,0.811119,...,66.482171,9.556019,26.801835,71.921117,76.936202,NaN,NaN,182.032889,172.692856,1235.959269
2025-01-14,13.687088,2.809349,1.968518,NaN,17.898716,36.120306,5.530420,1.135701,6.143860,0.828193,...,66.859292,9.477877,26.302304,74.280318,76.831920,NaN,NaN,179.167769,169.660334,1244.087975


In [4]:
returns = prices.iloc[:,1:].pct_change()

C:\Users\bapdu\AppData\Local\Temp\ipykernel_6784\701106193.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.iloc[:,1:].pct_change()


In [7]:
weights = pd.read_parquet('data/indices.parquet')
weights['Date'] = pd.to_datetime(weights['Date'])
weights = weights.set_index('Date')

In [10]:
aligned_prices = prices.reindex(weights.index)
aligned_returns = aligned_prices.pct_change()

C:\Users\bapdu\AppData\Local\Temp\ipykernel_6784\132733366.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  aligned_returns = aligned_prices.pct_change()


In [ ]:
common_tickers = aligned_returns.columns.intersection(weights.columns)
aligned_returns = aligned_returns[common_tickers]
weights = weights[common_tickers]

In [16]:
# Identifier les rendements non nuls
non_null_returns_mask = aligned_returns.isna() == False

# Calculer la somme des poids pour les rendements non nuls à chaque date
non_null_weights_sum = (weights * non_null_returns_mask).sum(axis=1)
non_null_weights_sum.iloc[0] = 1


In [37]:
weighted_returns = (aligned_returns * weights).sum(axis=1)/non_null_weights_sum
initial_index_value = 100  # Par exemple, l'indice commence à 100
index_values = (1 + weighted_returns).cumprod()

# Créer un DataFrame pour l'indice recomposé
index_df = pd.DataFrame({'Index Value': index_values})

In [44]:
(1+weighted_returns).cumprod()

Date
2007-04-30     1.000000
2007-05-31     1.047513
2007-06-29     1.040393
2007-07-31     1.011125
2007-08-31     1.021438
                ...    
2024-10-31    11.881305
2024-11-29    12.876252
2024-12-31    12.898131
2025-01-31    13.404818
2025-02-28    13.381199
Length: 215, dtype: float64

In [27]:
weighted_returns

Date
2007-04-30    0.000000
2007-05-31    0.047513
2007-06-29   -0.006797
2007-07-31   -0.028132
2007-08-31    0.010200
                ...   
2024-10-31    0.009183
2024-11-29    0.083740
2024-12-31    0.001699
2025-01-31    0.039284
2025-02-28   -0.001762
Length: 215, dtype: float64

In [26]:
index_values


Date
2007-04-30     1.000000
2007-05-31     1.047513
2007-06-29     1.040393
2007-07-31     1.011125
2007-08-31     1.021438
                ...    
2024-10-31    11.881305
2024-11-29    12.876252
2024-12-31    12.898131
2025-01-31    13.404818
2025-02-28    13.381199
Length: 215, dtype: float64

In [21]:
index_df

,Index Value
1 HK Equity,NaN
10 HK Equity,NaN
101 HK Equity,NaN
1031 HK Equity,NaN
1038 HK Equity,NaN
...,...
2024-10-31 00:00:00,NaN
2024-11-29 00:00:00,NaN
2024-12-31 00:00:00,NaN
2025-01-31 00:00:00,NaN


In [67]:
import plotly.graph_objects as go

# Tracer l'évolution de l'indice
fig = go.Figure()
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Créer une figure avec deux sous-graphiques (un pour chaque courbe)
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    subplot_titles=("Index Value Over Time", "Non-Null Weights Sum Over Time"))

# Ajouter la courbe de l'indice (premier graphique)
fig.add_trace(go.Scatter(
    x=index_df.index,
    y=index_df['Index Value'],
    mode='lines',
    name='Index Value',
    line=dict(color='blue')
), row=1, col=1)

# Ajouter la courbe de la somme des poids non nuls (deuxième graphique)
fig.add_trace(go.Scatter(
    x=non_null_weights_sum.index[1:],
    y=non_null_weights_sum[1:],
    mode='lines',
    name='Non-Null Weights Sum',
    line=dict(color='orange')
), row=2, col=1)

# Ajouter des titres et personnaliser le graphique
fig.update_layout(
    title='Index Value and Non-Null Weights Sum Over Time',
    xaxis_title='Date',
    yaxis_title='Value',
    height=600,  # Ajuster la hauteur de la figure
    template='plotly_white'
)

# Afficher le graphique
fig.show()

In [ ]:
### Save to parquet 
prices = pd.read_csv('data/msci_prices.csv')
prices = prices.rename(columns={'date': 'Date'})
prices.to_parquet('data/msci_prices.parquet', index=False)

In [76]:
weights = pd.read_excel('data/indices.xlsx', sheet_name='Composition MSCI World')
weights = weights.rename(columns={'Dates': 'Date'})
weights.to_parquet('data/indices.parquet', index=False)

In [77]:
weights = pd.read_parquet('data/indices.parquet')